<a href="https://colab.research.google.com/github/limkyuhee0/py/blob/master/CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIFAR-10 Classification
> I chose to use Greedy Algorithm over GridSearchCV algorithm for CIFAR-10 Image Classification. GridSearchCV might result in better performance but I believe it does not take much advantage of using GPU, and the time cost is higher than the advantage of convenience and accuracy. Therefore, I chose Greedy Algorithm for parameter settings. Given the range of values for a parameter, value that results in the best performance is chosen. The next parameter is chosen, assuming the value of the previously decided parameter as default. This process is done with 6 parameters, containing hyper-parameters.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.layers import Dropout
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Import Data & Check first 15 Pictures

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
plt.show()

In [ ]:
X_train = X_train.reshape((50000, 32, 32, 3,1))
X_test = X_test.reshape((10000, 32, 32, 3,1))
X_train , X_test = X_train/255, X_test/255

# Training Models

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hist = model.fit(X_train, y_train, epochs=10,batch_size=100, validation_data=(X_test,y_test))

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,10))
plt.plot(hist.history['val_accuracy'],label = 'test accuracy')
plt.plot(hist.history['accuracy'],label = 'train accuracy')
plt.legend()
plt.show()

## Dropout Dependencies

In [ ]:
dropouts = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
test_acc=[]
for dropout in dropouts:
    print('Dropout : ',(dropout))
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))


    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=100)
    loss,accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(dropouts, test_acc)
plt.xlabel('Dropout')
plt.ylabel('Accuracy')
plt.title('Dropout Decision')
plt.legend()
plt.show()

Therefore, drop out is set to 0.2 further on.

## Batch Size Dependencies

In [ ]:
batch_sizes = [50,100,200,500,1000]
dropout = 0.2
test_acc=[]
for batch_size in batch_sizes:
    print('batch_size : ',(batch_size))
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))


    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size)
    score, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(batch_sizes, test_acc)
plt.xlabel('Batch Size')
plt.ylabel('Accuracy')
plt.title('Batch Size Decision')
plt.legend()
plt.show()

Therefore, batch size is set to 50 further on.

## Kerner Initializer

In [ ]:
initializers = [tf.keras.initializers.Constant(3.),#Constant
                tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_in", distribution="truncated_normal", seed=None),#VarianceScaling
                tf.keras.initializers.VarianceScaling(scale=0.1, mode='fan_in', distribution='uniform'),#VarianceScaling
                tf.keras.initializers.GlorotNormal(),#GlorotNormal
                tf.keras.initializers.GlorotUniform(),#GlorotUniform
                tf.keras.initializers.HeNormal(),#HeNormal
                tf.keras.initializers.HeUniform(),#HeUniform
                tf.keras.initializers.Orthogonal()]#Orthogonal
batch_size = 50
dropout = 0.2
test_acc=[]
for initializer in initializers:
    print('initializer : ',(initializer))
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))


    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size)
    score, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(test_acc)
plt.xlabel('Initializers')
plt.ylabel('Accuracy')
plt.title('Initializers Decision')
plt.legend()
plt.show()

Therefore, "tf.keras.initializers.GlorotNormal()" kernel initializer is used further on

## Padding

In [ ]:
paddings = ['valid','same']
initializer = tf.keras.initializers.GlorotNormal()
batch_size = 50
dropout = 0.2
test_acc=[]
for padding in paddings:
    print('padding : ',padding)
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))


    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size)
    score, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(paddings, test_acc)
plt.xlabel('Paddings')
plt.ylabel('Accuracy')
plt.title('Paddings Decision')
plt.legend()
plt.show()

Therefore, padding is set to "same" further on.

## Neurons


In [ ]:
neurons = [[32,64,64,64],
           [32,64,128,32],
           [16,32,64,128]]
padding = "same"
initializer = tf.keras.initializers.HeUniform()
batch_size = 50
dropout = 0.2
test_acc=[]
for neuron in neurons:
    print('neuron : ',neuron)
    model = models.Sequential()

    model.add(layers.Conv2D(neuron[0], (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))


    model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(neuron[2], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Flatten())
    model.add(layers.Dense(neuron[3], activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size)
    score, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(test_acc)
plt.xlabel('Neurons')
plt.ylabel('Accuracy')
plt.title('Neurons Decision')
plt.legend()
plt.show()

Therefore, neurons for each layer is set to [32,64,128,32] further on.

## Epochs

In [ ]:
epochs=[10,20,50,100,200]
neuron = [32,64,128,32]
padding = "same"
initializer = tf.keras.initializers.HeUniform()
batch_size = 50
dropout = 0.2
test_acc=[]
for epoch in epochs:
    print('epoch : ',epoch)
    model = models.Sequential()

    model.add(layers.Conv2D(neuron[0], (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))


    model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(neuron[2], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Flatten())
    model.add(layers.Dense(neuron[3], activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=epoch,batch_size=batch_size)
    score, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(epochs, test_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Epochs Decision')
plt.legend()
plt.show()
print('Best Performance : ',max(test_acc))

Therefore, the best performance is 76.7% accuracy.

In [ ]:
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size = 0.1)

X_train = X_train.reshape((45000, 32, 32, 3,1))
X_val = X_val.reshape((5000, 32, 32, 3,1))
X_test = X_test.reshape((10000, 32, 32, 3,1))
X_train , X_val, X_test = X_train/255, X_val/255, X_test/255

# More Complicated CNNS with Validation

## Dropout Dependencies

In [ ]:
dropouts = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
test_acc=[]
for dropout in dropouts:
    print('Dropout : ',(dropout))
    
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=100,validation_data = (X_val, y_val))
    loss,accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(dropouts, test_acc)
plt.xlabel('Dropout')
plt.ylabel('Accuracy')
plt.title('Dropout Decision')
plt.legend()
plt.show()

Therefore, drop out is set to 0.2 further on.

## Batch Size Dependencies

In [ ]:
batch_sizes = [50,100,200,500,1000]
dropout = 0.2
test_acc=[]
for batch_size in batch_sizes:
    print('batch_size : ',(batch_size))
    
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size,validation_data = (X_val, y_val))
    loss,accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(batch_sizes, test_acc)
plt.xlabel('Batch Size')
plt.ylabel('Accuracy')
plt.title('Batch Size Decision')
plt.legend()
plt.show()

Therefore, batch size is set to 50 further on.

## Kerner Initializer

In [ ]:
initializers = [tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_in", distribution="truncated_normal", seed=None),#VarianceScaling
                tf.keras.initializers.VarianceScaling(scale=0.1, mode='fan_in', distribution='uniform'),#VarianceScaling
                tf.keras.initializers.GlorotNormal(),#GlorotNormal
                tf.keras.initializers.GlorotUniform(),#GlorotUniform
                tf.keras.initializers.HeNormal(),#HeNormal
                tf.keras.initializers.HeUniform(),#HeUniform
batch_size = 50
dropout = 0.2
test_acc=[]
for initializer in initializers:
    print('initializer : ',(initializer))  
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(layers.Conv2D(32, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size,validation_data = (X_val, y_val))
    loss,accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(test_acc)
plt.xlabel('Initializers')
plt.ylabel('Accuracy')
plt.title('Initializers Decision')
plt.legend()
plt.show()

Therefore, "tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_in", distribution="truncated_normal", seed=None)" kernel initializer is used further on

## Padding

In [ ]:
paddings = ['valid','same']
initializer = tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_in", distribution="truncated_normal", seed=None)
batch_size = 50
dropout = 0.2
test_acc=[]
for padding in paddings:
    print('padding : ',padding)
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(layers.Conv2D(32, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Flatten())
    model.add(layers.Dense(32, activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size,validation_data = (X_val, y_val))
    loss,accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.bar(paddings, test_acc)
plt.xlabel('Paddings')
plt.ylabel('Accuracy')
plt.title('Paddings Decision')
plt.legend()
plt.show()

Therefore, padding is set to "same" further on.

## Neurons


In [ ]:
neurons = [[32,64,64,64,128,128,32],
           [32,32,64,64,128,256,256],
           [32,64,128,256,256,256,512],
           [32,64,64,64,128,128,256],
           [64,64,128,128,256,256,32]]
padding = "same"
initializer = tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_in", distribution="truncated_normal", seed=None)
batch_size = 50
dropout = 0.2
test_acc=[]
for neuron in neurons:
    print('neuron : ',neuron)
    model = models.Sequential()

    model.add(layers.Conv2D(neuron[0], (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(neuron[2], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(layers.Conv2D(neuron[3], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(neuron[4], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(layers.Conv2D(neuron[5], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Flatten())
    model.add(layers.Dense(neuron[6], activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=10,batch_size=batch_size,validation_data = (X_val, y_val))
    loss,accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(test_acc)
plt.xlabel('Neurons')
plt.ylabel('Accuracy')
plt.title('Neurons Decision')
plt.legend()
plt.show()

Therefore, neurons for each layer is set to [32,64,128,32] further on.

## Epochs

In [ ]:
epochs=[10,20,50,100,200]
neuron = [32,64,128,32]
padding = "same"
initializer = tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_in", distribution="truncated_normal", seed=None)
batch_size = 50
dropout = 0.2
test_acc=[]
for epoch in epochs:
    print('epoch : ',epoch)
    model = models.Sequential()

    model.add(layers.Conv2D(neuron[0], (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Conv2D(neuron[2], (3, 3), activation='relu',kernel_initializer = initializer))
    model.add(Dropout(dropout))
    model.add(layers.MaxPooling2D((2, 2),padding = padding))

    model.add(layers.Flatten())
    model.add(layers.Dense(neuron[3], activation='relu',kernel_initializer = initializer))
    model.add(layers.Dense(10, activation='softmax'))

    # model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    hist = model.fit(X_train, y_train, epochs=epoch,batch_size=batch_size,validation_data = (X_val, y_val))
    score, accuracy = model.evaluate(X_test, y_test, verbose=0)
    test_acc.append(accuracy)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(epochs, test_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Epochs Decision')
plt.legend()
plt.show()
print('Best Performance : ',max(test_acc))

### Train, Val accuracy of the model with best performance(from above)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(hist.history['accuracy'],label='train_accuracy')
plt.plot(hist.history['val_accuracy'],label='val_accuracy')
plt.plot([accuracy]*100,label='test_accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Complicated CNN')
plt.legend()
plt.show()
print('Best Performance : ',max(test_acc))

## Changing Loss Function

In [ ]:
epochs=100
neuron = [32,64,128,32]
padding = "same"
initializer = tf.keras.initializers.HeUniform()
batch_size = 50
dropout = 0.2

#model

model = models.Sequential()

model.add(layers.Conv2D(neuron[0], (3, 3), activation='relu', input_shape=(32, 32, 3),kernel_initializer = initializer))
model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
model.add(Dropout(dropout))
model.add(layers.MaxPooling2D((2, 2),padding = padding))

model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
model.add(layers.Conv2D(neuron[1], (3, 3), activation='relu',kernel_initializer = initializer))
model.add(Dropout(dropout))
model.add(layers.MaxPooling2D((2, 2),padding = padding))

model.add(layers.Conv2D(neuron[2], (3, 3), activation='relu',kernel_initializer = initializer))
model.add(layers.Conv2D(neuron[2], (3, 3), activation='relu',kernel_initializer = initializer))
model.add(Dropout(dropout))
model.add(layers.MaxPooling2D((2, 2),padding = padding))

model.add(layers.Flatten())
model.add(layers.Dense(neuron[3], activation='relu',kernel_initializer = initializer))
model.add(layers.Dense(10, activation='softmax'))

# model.summary()

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

hist = model.fit(X_train, y_train, epochs=epochs,batch_size=batch_size,validation_data = (X_val, y_val))
score, accuracy = model.evaluate(X_test, y_test, verbose=0)